<a href="https://colab.research.google.com/github/sandhrabijoy/Pytorch-and-ML-learning/blob/master/MCQmodel_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install transformers torch nltk sentence-transformers

In [16]:
import random
import nltk
import torch
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util

In [17]:
nlk = nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
question_generator=pipeline('text2text-generation',model='valhalla/t5-base-qg-hl')
qa_pipeline=pipeline('question-answering',model='distilbert-base-cased-distilled-squad')
similarity_model=SentenceTransformer('all-MiniLM-L6-v2')

Device set to use cpu
Device set to use cpu


In [19]:
def generate_mcqs(text, num_questions=3):
    sentences = nltk.sent_tokenize(text)
    mcqs = []

    for _ in range(num_questions):
        # Select a random sentence
        sentence = random.choice(sentences)

        # Generate a question from the sentence
        question_input = f"highlight: {sentence} </s>"
        question = question_generator(question_input, max_length=64, do_sample=False)[0]["generated_text"]

        # Extract answer using QA model
        answer = qa_pipeline(question=question, context=sentence)["answer"]

        # Generate options (distractors)
        options = [answer] + [qa_pipeline(question="What is another topic with similar context of question?", context=s)["answer"] for s in sentences[:3]]
        options = list(set(filter(None, options)))  # Remove duplicates and empty values
        random.shuffle(options)

        mcqs.append({"question": question, "choices": options, "answer": answer})

    return mcqs


In [20]:
text_input=input("\nEnter the context:\n")


Enter the context:
The History and Impact of the Internet The Internet has transformed the world in ways unimaginable just a few decades ago. Originally developed as a military project, it has now become an essential part of daily life, enabling instant global communication, access to unlimited information, and new opportunities for businesses, education, and entertainment.  The origins of the Internet can be traced back to the late 1960s when the U.S. Department of Defense developed ARPANET (Advanced Research Projects Agency Network). The goal of ARPANET was to create a decentralized network that could continue functioning even if parts of it were destroyed. This innovation introduced the concept of packet switching, allowing data to be sent in small chunks and reassembled at its destination.  By the 1980s, ARPANET had evolved into a larger system that connected various universities and research centers. The introduction of the Domain Name System (DNS) in 1983 simplified web navigati

In [23]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [24]:
mcq_questions=generate_mcqs(text_input)

In [25]:
for idx, mcq in enumerate(mcq_questions, 1):
    print(f"\nQ{idx}: {mcq['question']}")
    for i, choice in enumerate(mcq['choices'], 1):
        print(f"   {i}. {choice}")
    print(f"Answer: {mcq['answer']}\n")


Q1: What was the name of the domain name system introduced in 1983?
   1. education, and entertainment
   2. Domain Name System
   3. Internet
   4. The History and Impact of the Internet
Answer: Domain Name System


Q2: What was the goal of ARPANET?
   1. The History and Impact of the Internet
   2. to create a decentralized network that could continue functioning
   3. Internet
   4. education, and entertainment
Answer: to create a decentralized network that could continue functioning


Q3: What is the main benefit of the Internet?
   1. Internet
   2. education, and entertainment
   3. instant global communication, access to unlimited information
   4. The History and Impact of the Internet
Answer: instant global communication, access to unlimited information

